In [929]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

from sklearn.metrics import f1_score, confusion_matrix, classification_report
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, StratifiedKFold

import os
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import random
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings(action='ignore') 

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [930]:
file_version = 54
CFG = {
    'EPOCHS': 30,
    'LEARNING_RATE':1e-2,    # 0.01
    'BATCH_SIZE':256,  #256
    'SEED':43,
    'log_transform_turn': 1,  # 로그 트랜스폼을 몇번 돌릴 것인가?
    'skew_cut':0.01,           # 왜도 얼마 이상을 로그값으로 정규화할 것인가?(100이면 안하겠다는 것)
    'vif_cut': 10,           # 다중공선성 vif 얼마 이상을 제거할 것인가?
    'corr_cut': 0.004,     # 일정 상관관계 이하 칼럼은 드랍
    'outlier_corr_cut': 100,    # 이값 이상의 상관관계를 갖는 칼럼에 대해서만 아웃라이어 제거(100이면 미적용)
    'T_Thresh': 0.3,     # Teacher model train Threshhold 최초 0.35
    'S_Thresh': 0.325,     # Student model train Threshold 최초 0.35
    'reduct' : 'sum',      # sum, mean
    'drop_rate': 0.2,
    
}

In [931]:
import pickle
#불러오기

with open(f"./data/{file_version}_1_train_X", 'rb') as pickle_filename:
    train_X_1 = pickle.load(pickle_filename)
with open(f"./data/{file_version}_1_train_y", 'rb') as pickle_filename:
    train_y_1 = pickle.load(pickle_filename)
with open(f"./data/{file_version}_1_val_X", 'rb') as pickle_filename:
    val_X_1 = pickle.load(pickle_filename)
with open(f"./data/{file_version}_1_val_y", 'rb') as pickle_filename:
    val_y_1 = pickle.load(pickle_filename)     
with open(f"./data/{file_version}_1_test", 'rb') as pickle_filename:
    test_1 = pickle.load(pickle_filename)

with open(f"./data/{file_version}_2_train_X", 'rb') as pickle_filename:
    train_X_2 = pickle.load(pickle_filename)
with open(f"./data/{file_version}_2_train_y", 'rb') as pickle_filename:
    train_y_2 = pickle.load(pickle_filename)
with open(f"./data/{file_version}_2_val_X", 'rb') as pickle_filename:
    val_X_2 = pickle.load(pickle_filename)
with open(f"./data/{file_version}_2_val_y", 'rb') as pickle_filename:
    val_y_2 = pickle.load(pickle_filename)     
with open(f"./data/{file_version}_2_test", 'rb') as pickle_filename:
    test_2 = pickle.load(pickle_filename)

with open(f"./data/{file_version}_3_train_X", 'rb') as pickle_filename:
    train_X_3 = pickle.load(pickle_filename)
with open(f"./data/{file_version}_3_train_y", 'rb') as pickle_filename:
    train_y_3 = pickle.load(pickle_filename)
with open(f"./data/{file_version}_3_val_X", 'rb') as pickle_filename:
    val_X_3 = pickle.load(pickle_filename)
with open(f"./data/{file_version}_3_val_y", 'rb') as pickle_filename:
    val_y_3 = pickle.load(pickle_filename)     
with open(f"./data/{file_version}_3_test", 'rb') as pickle_filename:
    test_3 = pickle.load(pickle_filename)
    
with open(f"./data/{file_version}_4_train_X", 'rb') as pickle_filename:
    train_X_4 = pickle.load(pickle_filename)
with open(f"./data/{file_version}_4_train_y", 'rb') as pickle_filename:
    train_y_4 = pickle.load(pickle_filename)
with open(f"./data/{file_version}_4_val_X", 'rb') as pickle_filename:
    val_X_4 = pickle.load(pickle_filename)
with open(f"./data/{file_version}_4_val_y", 'rb') as pickle_filename:
    val_y_4 = pickle.load(pickle_filename)     
with open(f"./data/{file_version}_4_test", 'rb') as pickle_filename:
    test_4 = pickle.load(pickle_filename)

FileNotFoundError: [Errno 2] No such file or directory: './data/53_1_train_X'

In [ ]:
train_X_list = [train_X_1, train_X_2, train_X_3, train_X_4]
train_y_list = [train_y_1, train_y_2, train_y_3, train_y_4]
val_X_list = [val_X_1, val_X_2, val_X_3, val_X_4]
val_y_list = [val_y_1, val_y_2, val_y_3, val_y_4]
test_list = [test_1, test_2, test_3, test_4]

In [ ]:
train_X_1.shape, train_y_1.shape, val_X_1.shape, val_y_1.shape

In [ ]:
# test_1.iloc[:,:-1]

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, data_X, data_y, distillation=False):
        super(CustomDataset, self).__init__()
        self.data_X = data_X
        self.data_y = data_y
        self.distillation = distillation
        
    def __len__(self):
        return len(self.data_X)
    
    def __getitem__(self, index):
        if self.distillation:
            # 지식 증류 학습 시
            teacher_X = torch.Tensor(self.data_X.iloc[index])
            student_X = torch.Tensor(self.data_X[test_stage_features].iloc[index])
            y = self.data_y.values[index]
            return teacher_X, student_X, y
        else:
            if self.data_y is None:
                test_X = torch.Tensor(self.data_X.iloc[index])
                return test_X
            else:
                teacher_X = torch.Tensor(self.data_X.iloc[index])

                y = self.data_y.values[index]
                return teacher_X, y

In [ ]:
train_loaders = []
val_loaders = []

for a, b, c, d, e in zip(train_X_list, train_y_list, val_X_list, val_y_list, test_list):
    
    test_stage_features = e.iloc[:,:-1].columns
    
    train_dataset = CustomDataset(a, b, False)
    val_dataset = CustomDataset(c, d, False)
    
    train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False)
    
#     print(train_loader)
#     print(list(train_loader))
    
    train_loaders.append(train_loader)
    val_loaders.append(val_loader)


In [ ]:
class Teacher(nn.Module):
    def __init__(self):
        super(Teacher, self).__init__()
        self.classifier = nn.Sequential(
            nn.Linear(in_features=train_col_num, out_features=256),
            nn.Dropout(p=CFG['drop_rate']),
            nn.BatchNorm1d(256),
            nn.LeakyReLU(),
            nn.Linear(in_features=256, out_features=1024),
            nn.Dropout(p=CFG['drop_rate']),
            nn.BatchNorm1d(1024),
            nn.LeakyReLU(),
            nn.Linear(in_features=1024, out_features=256),
            nn.Dropout(p=CFG['drop_rate']),
            nn.BatchNorm1d(256),
            nn.LeakyReLU(),
            nn.Linear(in_features=256, out_features=1),
            nn.Sigmoid()
        )
        
    def forward(self, x):
        output = self.classifier(x)
        return output

In [ ]:
def train(model, optimizer, train_loader, val_loader, scheduler, device):
    model.to(device)

    best_score = 0
    best_model = None
    
    # Loss Function 정의
    criterion = nn.BCELoss(reduction = CFG['reduct']).to(device)  #BCEWithLogitsLoss
#     criterion = nn.BCEWithLogitsLoss(reduction = CFG['reduct']).to(device)  #BCEWithLogitsLoss

    for epoch in range(CFG["EPOCHS"]):
        train_loss = []
  
        model.train()
        for X, y in tqdm(train_loader):
            X = X.float().to(device)
            y = y.float().to(device)
            
            optimizer.zero_grad()
            
            y_pred = model(X)
            
            loss = criterion(y_pred, y.reshape(-1, 1))  # loss_fn(input, target)
            loss.backward()
            
            optimizer.step()

            train_loss.append(loss.item())

        val_loss, val_score = validation_teacher(model, val_loader, criterion, device)
        print(f'Epoch [{epoch}], Train Loss : [{np.mean(train_loss) :.5f}] Val Loss : [{np.mean(val_loss) :.5f}] Val F1 Score : [{val_score:.5f}]')

        if scheduler is not None:
            scheduler.step(val_score)
            
        if best_score < val_score:
            best_model = model
            best_score = val_score
    print(f"Teacher Train Best Score는 {best_score} 입니다.")
    return best_model 

In [ ]:
def competition_metric(true, pred):
    return f1_score(true, pred, average="macro")


def validation_teacher(model, val_loader, criterion, device):
    model.eval()

    val_loss = []
    pred_labels = []
    true_labels = []
    threshold = CFG['T_Thresh']     
    
    with torch.no_grad():
        for X, y in tqdm(val_loader):
            X = X.float().to(device)
            y = y.float().to(device)
            
            model_pred = model(X.to(device))
            
            loss = criterion(model_pred, y.reshape(-1, 1))
            val_loss.append(loss.item())      
            
            model_pred = model_pred.squeeze(1).to('cpu')  # squeeze함수는 차원이 1인 차원을 제거해준다.
            pred_labels += model_pred.tolist()
            true_labels += y.tolist()
        
        #print(pred_labels[0:10])
        # 오일상태는 0: 정상, 1: 이상
        # 어느 임계점 thrshold보다 큰 예측 라벨을 1로 하고, 나머지를 0으로 할 것인가?
        pred_labels = np.where(np.array(pred_labels) > threshold, 1, 0)
        #print(pred_labels[0:10])
        cf = confusion_matrix(true_labels, pred_labels)
        val_f1 = competition_metric(true_labels, pred_labels)
        
        print(cf)
        print(classification_report(true_labels, pred_labels, target_names=["0: 정상", "1: 이상"]))
    return val_loss, val_f1

In [ ]:
teachers = []

for a, b, c, d in zip(train_loaders, val_loaders, train_X_list, test_list):
    train_col_num = len(c.columns)
    test_stage_features = d.iloc[:,:-1].columns
    test_col_num = len(d.columns)
    
    print("***")
    
    model = Teacher()
    model.eval()
    optimizer = torch.optim.Adam(model.parameters(), lr=CFG['LEARNING_RATE'])
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=1, threshold_mode='abs',min_lr=1e-8, verbose=True)
    pred_labels = []
    true_labels = []

    teacher_model = train(model, optimizer, a, b, scheduler, device)
    teachers.append(teacher_model)
    
    print("부분학습 종료")
    
    
print("전체 학습 종료")

In [ ]:
for i, teacher in enumerate(teachers):
    PATH1 = f"./models/{file_version}_{i}_teacher_net.pth"
    torch.save(teacher.state_dict(), PATH1)

In [ ]:
class Student(nn.Module):
    def __init__(self):
        super(Student, self).__init__()
        self.classifier = nn.Sequential(
            nn.Linear(in_features=test_col_num, out_features=128),
            nn.Dropout(p=CFG['drop_rate']),
            nn.BatchNorm1d(128),
            nn.LeakyReLU(),
            nn.Linear(in_features=128, out_features=512),
            nn.Dropout(p=CFG['drop_rate']),
            nn.BatchNorm1d(512),
            nn.LeakyReLU(),
            nn.Linear(in_features=512, out_features=128),
            nn.Dropout(p=CFG['drop_rate']),
            nn.BatchNorm1d(128),
            nn.LeakyReLU(),
            nn.Linear(in_features=128, out_features=1),
            nn.Sigmoid()
        )
        
    def forward(self, x):
        output = self.classifier(x)
        return output

In [938]:
# distillation Function 정의

# def distillation(student_logits, labels, teacher_logits, alpha, T):
def distillation(student_logits, labels, teacher_logits, alpha):

    student_loss = nn.BCELoss(reduction = CFG['reduct'])(student_logits, labels.reshape(-1, 1)) #BCEWithLogitsLoss
#     student_loss = nn.BCEWithLogitsLoss(reduction = CFG['reduct'])(student_logits, labels.reshape(-1, 1)) #BCEWithLogitsLoss
    
    distillation_loss = nn.BCELoss()(student_logits, teacher_logits)
#     distillation_loss = nn.KLDivLoss(reduction='batchmean')(F.log_softmax(student_logits/T, dim=1), F.softmax(teacher_logits/T, dim=1)) * (T * T)
    
    return alpha * student_loss + (1-alpha) * distillation_loss

def distill_loss(output, target, teacher_output, loss_fn=distillation, opt=optimizer):
#     loss_b = loss_fn(output, target, teacher_output, alpha=0.1, T=10)
    loss_b = loss_fn(output, target, teacher_output, alpha=0.1)

    if opt is not None:
        opt.zero_grad()
        loss_b.backward()
        opt.step()

    return loss_b.item()

In [939]:
def student_train(s_model, t_model, optimizer, train_loader, val_loader, scheduler, device):
    s_model.to(device)
    t_model.to(device)
    
    best_score = 0
    best_model = None

    for epoch in range(CFG["EPOCHS"]):
        train_loss = []
        s_model.train()
        t_model.eval()    # teather는 이번에는 학습하지 않으므로 .eval()로 불러온다
        
        for X_t, X_s, y in tqdm(train_loader):
            X_t = X_t.float().to(device)
            X_s = X_s.float().to(device)
            y = y.float().to(device)
            
            optimizer.zero_grad()

            output = s_model(X_s)
            with torch.no_grad():
                teacher_output = t_model(X_t)
                
            loss_b = distill_loss(output, y, teacher_output, loss_fn=distillation, opt=optimizer)

            train_loss.append(loss_b)

        val_loss, val_score = validation_student(s_model, t_model, val_loader, distill_loss, device)
        print(f'Epoch [{epoch}], Train Loss : [{np.mean(train_loss) :.5f}] Val Loss : [{np.mean(val_loss) :.5f}] Val F1 Score : [{val_score:.5f}]')
        
        if scheduler is not None:
            scheduler.step(val_score)
            
        if best_score < val_score:
            best_model = s_model
            best_score = val_score
    
    print(f"Student Train Best Score는 {best_score} 입니다.")
        
    return best_model

In [940]:
def validation_student(s_model, t_model, val_loader, criterion, device):
    s_model.eval()
    t_model.eval()

    val_loss = []
    pred_labels = []
    true_labels = []
    threshold = CFG['S_Thresh']
    
    with torch.no_grad():
        for X_t, X_s, y in tqdm(val_loader):
            X_t = X_t.float().to(device)
            X_s = X_s.float().to(device)
            y = y.float().to(device)
            
            model_pred = s_model(X_s)
            teacher_output = t_model(X_t)
            
            loss_b = distill_loss(model_pred, y, teacher_output, loss_fn=distillation, opt=None)
            val_loss.append(loss_b)
            
            model_pred = model_pred.squeeze(1).to('cpu')
            pred_labels += model_pred.tolist()
            true_labels += y.tolist()
        
        pred_labels = np.where(np.array(pred_labels) > threshold, 1, 0)
        val_f1 = competition_metric(true_labels, pred_labels)
        cf = confusion_matrix(true_labels, pred_labels)        
        print(cf)
        print(classification_report(true_labels, pred_labels, target_names=["0: 정상", "1: 이상"]))
        
    return val_loss, val_f1    

In [941]:
s_train_loaders = []
s_val_loaders = []

for a, b, c, d, e in zip(train_X_list, train_y_list, val_X_list, val_y_list, test_list):
    
    test_stage_features = e.iloc[:,:-1].columns
    
    train_dataset = CustomDataset(a, b, True)
    val_dataset = CustomDataset(c, d, True)
    
    train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False)
    
#     print(train_loader)
#     print(list(train_loader))
    
    s_train_loaders.append(train_loader)
    s_val_loaders.append(val_loader)


In [942]:
students = []

for a, b, c, d, e in zip(s_train_loaders, s_val_loaders, train_X_list, test_list, teachers):
    train_col_num = len(c.columns)
    test_stage_features = d.iloc[:,:-1].columns
#     print(test_stage_features)
    test_col_num = len(test_stage_features)

    student_model = Student()
    student_model.eval()
    optimizer = torch.optim.Adam(student_model.parameters(), lr=CFG['LEARNING_RATE'])
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=1, threshold_mode='abs',min_lr=1e-8, verbose=True)

    best_student_model = student_train(student_model, e, optimizer, a, b, scheduler, device)

    students.append(best_student_model)
    
    print("부분학습 종료")

print("전체 학습 종료")
    
# print(students)

  0%|          | 0/13 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

[[653  63]
 [ 51  11]]
              precision    recall  f1-score   support

       0: 정상       0.93      0.91      0.92       716
       1: 이상       0.15      0.18      0.16        62

    accuracy                           0.85       778
   macro avg       0.54      0.54      0.54       778
weighted avg       0.87      0.85      0.86       778

Epoch [0], Train Loss : [9.26073] Val Loss : [6.28388] Val F1 Score : [0.54074]


  0%|          | 0/13 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

[[143  15]
 [  8   2]]
              precision    recall  f1-score   support

       0: 정상       0.95      0.91      0.93       158
       1: 이상       0.12      0.20      0.15        10

    accuracy                           0.86       168
   macro avg       0.53      0.55      0.54       168
weighted avg       0.90      0.86      0.88       168

Epoch [28], Train Loss : [2.48474] Val Loss : [4.86479] Val F1 Score : [0.53686]


  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

[[143  15]
 [  8   2]]
              precision    recall  f1-score   support

       0: 정상       0.95      0.91      0.93       158
       1: 이상       0.12      0.20      0.15        10

    accuracy                           0.86       168
   macro avg       0.53      0.55      0.54       168
weighted avg       0.90      0.86      0.88       168

Epoch [29], Train Loss : [2.40450] Val Loss : [4.85624] Val F1 Score : [0.53686]
Epoch 00030: reducing learning rate of group 0 to 2.4414e-06.
Student Train Best Score는 0.6108859293572669 입니다.
부분학습 종료
전체 학습 종료


In [943]:
def choose_threshold(model, val_loader, device):
    model.to(device)
    model.eval()
    
#     thresholds = [0.1, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5]
    thresholds = [0.05, 0.1, 0.15, 0.2, 0.25, 0.3]

    pred_labels = []
    true_labels = []
    
    best_score = 0
    best_thr = None
    with torch.no_grad():
        for _, x_s, y in tqdm(iter(val_loader)):
            x_s = x_s.float().to(device)
            y = y.float().to(device)
            
            model_pred = model(x_s)
            
            model_pred = model_pred.squeeze(1).to('cpu')
            pred_labels += model_pred.tolist()
            true_labels += y.tolist()
        
        for threshold in thresholds:
            pred_labels_thr = np.where(np.array(pred_labels) > threshold, 1, 0)
            score_thr = competition_metric(true_labels, pred_labels_thr)
            cf1 = confusion_matrix(true_labels, pred_labels_thr)  

#             print(cf1)
#             print(classification_report(true_labels, pred_labels_thr, target_names=["0: 정상", "1: 이상"]))

            if best_score < score_thr:
                best_score = score_thr
                best_thr = threshold
                
    return best_thr, best_score

In [944]:
# for a, b in zip(students, s_val_loaders):
    
#     print(list(b))
    
for a, b, c, d, e, f in zip(s_train_loaders, s_val_loaders, train_X_list, test_list, teachers, students):
    train_col_num = len(c.columns)
    test_stage_features = d.iloc[:,:-1].columns
    test_col_num = len(test_stage_features)


    choose_threshold(f, b, device)
    best_threshold, best_score = choose_threshold(f, b, device)
    choose_threshold_result = f'Best Threshold : [{best_threshold}], Score : [{best_score:.5f}]'
    print(choose_threshold_result)
    
    print("부분 작업 종료")

print("전체 작업 종료")

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

Best Threshold : [0.3], Score : [0.53977]
부분 작업 종료


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

...

  0%|          | 0/6 [00:00<?, ?it/s]

Best Threshold : [0.3], Score : [0.58700]
부분 작업 종료


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Best Threshold : [0.25], Score : [0.56078]
부분 작업 종료
전체 작업 종료


In [945]:
test_2

,encoder__x1_2008,encoder__x1_2009,encoder__x1_2010,encoder__x1_2011,encoder__x1_2012,encoder__x1_2013,encoder__x1_2014,encoder__x1_2015,encoder__x1_2016,encoder__x1_2017,...,FE,H2O,MN,MO,NI,TI,V,V40,ZN,order
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.294593,-0.077342,-0.352471,-0.375764,-0.144333,-0.131016,-0.083807,-0.737903,1.041387,2.0
1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.432480,-0.077342,-0.352471,-0.375764,-0.144333,-0.131016,-0.083807,0.986401,-0.183985,4.0
2,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,-0.294593,-0.077342,-0.352471,0.901335,-0.144333,-0.131016,-0.083807,0.736600,-0.279014,19.0
3,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.143751,-0.077342,-0.352471,-0.375764,-0.144333,-0.131016,-0.083807,-0.684101,0.591250,21.0
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.695458,-0.077342,-0.352471,-0.375764,-0.144333,-0.131016,-0.083807,0.388139,0.031080,38.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
988,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.491684,-0.077342,-0.352471,-0.375764,-0.144333,7.917125,-0.083807,0.598527,0.176124,6017.0
989,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,-0.294593,-0.077342,-0.352471,-0.375764,-0.144333,-0.131016,-0.083807,-0.737903,0.676276,6020.0
990,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.694531,-0.077342,-0.352471,-0.375764,-0.144333,-0.131016,-0.083807,0.040763,0.431202,6021.0
991,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.938797,-0.077342,2.186212,-0.375764,-0.144333,-0.131016,-0.083807,-1.360026,-0.404052,6028.0


In [946]:
test_loaders = []

for i in test_list:

    test_dataset = CustomDataset(i.iloc[:,:-1], None, False)
    test_loader = DataLoader(test_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False)
    test_loaders.append(test_loader)

# print(list(test_loaders[0]))
    

[tensor([[ 0.0000,  0.0000,  0.0000,  ..., -0.1559, -0.8498, -0.3947],
        [ 0.0000,  0.0000,  0.0000,  ..., -0.1559,  1.7224,  0.1522],
        [ 0.0000,  0.0000,  0.0000,  ..., -0.1559,  1.1441, -0.9415],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ..., -0.1559,  1.1309, -0.9697],
        [ 0.0000,  0.0000,  0.0000,  ..., -0.1559, -0.2682, -0.7048],
        [ 0.0000,  0.0000,  0.0000,  ..., -0.1559, -0.2210, -1.0768]]), tensor([[ 0.0000,  0.0000,  0.0000,  ..., -0.1559, -0.4844, -0.2368],
        [ 0.0000,  0.0000,  0.0000,  ..., -0.1559,  0.6963,  1.3643],
        [ 0.0000,  0.0000,  0.0000,  ..., -0.1559,  0.4697,  2.0351],
        ...,
        [ 0.0000,  0.0000,  1.0000,  ..., -0.1559,  0.5909,  1.0204],
        [ 0.0000,  1.0000,  0.0000,  ..., -0.1559, -0.4600, -0.0903],
        [ 0.0000,  0.0000,  0.0000,  ..., -0.1559,  0.4047, -0.5018]]), tensor([[ 0.0000,  0.0000,  0.0000,  ..., -0.1559, -0.0502,  0.3495],
        [ 0.0000,  0.0000,  0.0000,  ..., -0.1559,  0.9775,

In [947]:
# test_datasets = CustomDataset(test, None, False)
# test_loaders = DataLoader(test_datasets, batch_size = CFG['BATCH_SIZE'], shuffle=False)

In [948]:
def inference(model, test_loader, threshold, device):
    model.to(device)
    model.eval()
    # .eval함수는 evaluation 과정에서 사용하지 말아야 하는 layer들을 알아서 off 시키는 함수
    
    test_predict = []
    with torch.no_grad():
        for x in tqdm(test_loader):
            x = x.float().to(device)
            model_pred = model(x)

            model_pred = model_pred.squeeze(1).to('cpu')
            test_predict += model_pred
        
    test_predict = np.where(np.array(test_predict) > threshold, 1, 0)  
    # threshhold 보다 큰 값을 찾아서 1로 바꾸고, 아닌 것은 0으로 변경해라
    print('Done.')
    return test_predict

In [949]:
pred_list = []
order_list = []
for a, b, c in zip(students, test_loaders, test_list):
    preds = inference(a, b, best_threshold, device)
    print(len(preds))
    print(len(c.order.tolist()))
    pred_list.extend(preds)
    order_list.extend(c.order.tolist())
print(len(pred_list))
print(len(order_list))
    

  0%|          | 0/7 [00:00<?, ?it/s]

Done.
1667
1667


  0%|          | 0/4 [00:00<?, ?it/s]

Done.
993
993


  0%|          | 0/12 [00:00<?, ?it/s]

Done.
3021
3021


  0%|          | 0/2 [00:00<?, ?it/s]

Done.
360
360
6041
6041


In [950]:
{"예측값": pred_list, "순서": order_list}

{'예측값': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0

In [951]:
result = pd.DataFrame({"예측값": pred_list, "순서": order_list})
result.sort_values(by=["순서"], inplace=True)
result

,예측값,순서
0,0,0.0
2660,0,1.0
1667,0,2.0
2661,1,3.0
1668,0,4.0
...,...,...
5678,0,6036.0
5679,0,6037.0
5680,0,6038.0
2659,0,6039.0


In [952]:
final_result = result["예측값"].tolist()
final_result

[0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,


In [953]:

# preds = inference(best_student_model, test_loaders, best_threshold, device)

In [954]:
submit = pd.read_csv('./data/sample_submission.csv')
submit['Y_LABEL'] = final_result
submit.head()

,ID,Y_LABEL
0,TEST_0000,0
1,TEST_0001,0
2,TEST_0002,0
3,TEST_0003,1
4,TEST_0004,0


In [955]:
pred_cnt = tuple(submit["Y_LABEL"].value_counts())
a, b = pred_cnt
r = f"정상값 개수 : {a}, 이상값 개수 : {b}"
print(r)

정상값 개수 : 5386, 이상값 개수 : 655


In [956]:
submit.to_csv(f'./submits/submit{file_version}.csv', index=False)

In [957]:
record = {"ver": file_version, "CFG": CFG, "Val_result": choose_threshold_result, "pred_cnt": r}
record

{'ver': 53,
 'CFG': {'EPOCHS': 30,
  'LEARNING_RATE': 0.01,
  'BATCH_SIZE': 256,
  'SEED': 43,
  'log_transform_turn': 1,
  'skew_cut': 0.01,
  'vif_cut': 10,
  'corr_cut': 0.004,
  'outlier_corr_cut': 100,
  'T_Thresh': 0.3,
  'S_Thresh': 0.325,
  'reduct': 'sum',
  'drop_rate': 0.2},
 'Val_result': 'Best Threshold : [0.25], Score : [0.56078]',
 'pred_cnt': '정상값 개수 : 5386, 이상값 개수 : 655'}